# cooking the best possible web search result (in termes of relevance and speed)

In [1]:
user_query = "what is the latest top loser in the American stock market"

## test functions

In [ ]:
from agentic_search.functions.web import get_serp_links

links = await get_serp_links(user_query)
links


In [3]:
from agentic_search.functions.web import get_webpages_soups_text

res = get_webpages_soups_text([x["href"] for x in links], user_query)
res

## test chains

In [ ]:
from agentic_search.chains.web import get_web_search_queries_chain

chain = get_web_search_queries_chain()
res = chain.invoke({"query": user_query})
res


## test tools

In [ ]:
from agentic_search.tools.web import get_agentic_web_search_results_tool

res = await get_agentic_web_search_results_tool(user_query)
# write results to file for consultation
with open("scratchpad.md", "w") as f:
    f.write(res)

## test capabilities

In [2]:
from agentic_search.capabilities.web import get_web_search_report

res = await get_web_search_report(user_query)
res


[DEBUG] 
invoking web search agent with messages: [SystemMessage(content='You are a precise research assistant with web search capabilities. Your tasks:\n1. Provide accurate, current information\n2. Synthesize multi-source information concisely\n3. Include citations and maintain objectivity\n\nFocus on authoritative, verifiable sources only.\n\nSkip web search if you are completely certain of information.\nOtherwise, perform targeted searches as needed.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the latest top loser in the American stock market', additional_kwargs={}, response_metadata={}, id='7ae9549b-3474-42c4-bcd7-3b231fdd4713')]

[DEBUG] 
invoking web search tool with query: latest top loser in American stock market

[DEBUG] 
Search attempt failed: https://links.duckduckgo.com/d.js?q=latest+top+performing+stocks+in+American+stock+market&kl=wt-wt&l=wt-wt&p=&s=0&df=&vqd=4-307563389108654770463578618576759759425&bing_market=wt-WT&ex=-1 202 Ratelimit



"Based on the provided content, here's a Markdown-formatted summary that answers the query:\n\n**Latest Top Loser in American Stock Market**\n\nThe latest top loser in the American stock market is Volcan Inc, with a decline of -99.78% in 2024 year-to-date."

In [3]:
# write results to file for consultation
with open("scratchpad.md", "w") as f:
    f.write(res)